In [1]:
import datasets
from tqdm.notebook import tqdm
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient, models, conversions
from langchain.docstore.document import Document as LangchainDocument

In [2]:
client = QdrantClient(path="medicalqna.db")

In [3]:
encoder = SentenceTransformer("all-MiniLM-L6-v2")

/workspace/.pyenv_mirror/user/current/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
client.recreate_collection(
    collection_name="medicalqna",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),  # Vector size is defined by used model
        distance=models.Distance.COSINE,
    ),
)

/tmp/ipykernel_5213/3471414559.py:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [5]:
ds = datasets.load_dataset("ruslanmv/ai-medical-chatbot", split="train")

In [6]:
ds

Dataset({
    features: ['Description', 'Patient', 'Doctor'],
    num_rows: 256916
})

In [8]:
RAW_KNOWLEDGE_BASE = [
    LangchainDocument(page_content=doc["Description"], metadata={"question": doc["Patient"], "answer": doc["Doctor"]})
    for doc in tqdm(ds)
]

  0%|          | 0/256916 [00:00<?, ?it/s]

In [9]:
points = []
for idx, doc in tqdm(enumerate(RAW_KNOWLEDGE_BASE), total=len(RAW_KNOWLEDGE_BASE)):
    content = f'''{doc.page_content}: {doc.metadata["question"]}: {doc.metadata["answer"]}'''
    vector = encoder.encode(content).tolist()
    payload = {"question": doc.metadata["question"], "answer": doc.metadata["answer"], "page_content": doc.page_content}
    point = models.PointStruct(id=idx, vector=vector, payload=payload)
    points.append(point)

  0%|          | 0/256916 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
points[0]

PointStruct(id=0, vector=[-0.004023421090096235, -0.005357899237424135, 0.06804246455430984, 0.066102996468544, -0.07491514086723328, -0.0009945608908310533, -0.011394156143069267, 0.08904100209474564, -0.024408191442489624, 0.04369188845157623, 0.02114597149193287, 0.10576315969228745, -0.021432047709822655, 0.0069670709781348705, -0.05428933724761009, 0.06488518416881561, -0.040500059723854065, -0.04325779527425766, -0.018332326784729958, 0.0813630223274231, 0.0723172053694725, 0.01802097074687481, -0.013998948968946934, -0.012376553378999233, -0.07446275651454926, 0.015276233665645123, -0.008094154298305511, -0.029375070706009865, 0.09893211722373962, -0.01204764936119318, 0.021430907770991325, 0.05923422425985336, -0.01596364565193653, -0.0030213678255677223, -0.0162824559956789, 0.0483940914273262, -0.16837115585803986, -0.02928023971617222, -0.05569662153720856, -0.017170332372188568, 0.056520309299230576, -0.03506201505661011, 0.013154755346477032, 0.018889956176280975, 0.043197

In [ ]:
client.upload_records(
    collection_name="medicalqna",
    records=points,
    parallel=8
)

/tmp/ipykernel_3870611/1009793423.py:1: DeprecationWarning: `upload_records` is deprecated, use `upload_points` instead
  client.upload_records(


In [11]:
hits = client.search(
    collection_name="medicalqna",
    query_vector=encoder.encode("how to perform cpr?").tolist(),
    limit=5
)

In [13]:
context = ""
for hit in hits:
    context += hit.payload['answer']

In [14]:
context

"Cardiac arrest requires immediate CPR if necessary ACLS or ALS(Advanced Life Support).  However in case of Bronchial Asthma if a patients develop Respiratory fatigue and his Oxygen concentration in blood starts falling put him on a ventilator, but wean him off as early as possible OR if he develops Cyanosis CPR + ALS.  There are other protocols for all the above disease but you stick to cardiac arrest and asthma.  Don't forget Cardiac Asthma, severe cases may requires ACLS.  So it is the indication and symtomology of the patient that gives rise to CPR, ACLS etc. no hard and fast rule.Hi. First of all, I appreciate your interest to know the emergency life-saving procedure. Tongue bite usually occurs in seizures and not in heart attacks. There is a possibility of a seizure in her case. Many reasons can be there for such an incident. It can be due to fall in sugar levels, due to side effect of asthalin in medicines, due to missed drug, due to high potassium caused by respiratory drugs, d

In [15]:
for hit in hits:
    print(hit.payload['answer'])

Cardiac arrest requires immediate CPR if necessary ACLS or ALS(Advanced Life Support).  However in case of Bronchial Asthma if a patients develop Respiratory fatigue and his Oxygen concentration in blood starts falling put him on a ventilator, but wean him off as early as possible OR if he develops Cyanosis CPR + ALS.  There are other protocols for all the above disease but you stick to cardiac arrest and asthma.  Don't forget Cardiac Asthma, severe cases may requires ACLS.  So it is the indication and symtomology of the patient that gives rise to CPR, ACLS etc. no hard and fast rule.
Hi. First of all, I appreciate your interest to know the emergency life-saving procedure. Tongue bite usually occurs in seizures and not in heart attacks. There is a possibility of a seizure in her case. Many reasons can be there for such an incident. It can be due to fall in sugar levels, due to side effect of asthalin in medicines, due to missed drug, due to high potassium caused by respiratory drugs, d